# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXDI1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [4]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group", 
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

Interclass correlation (ICC) is a measure of how similar the values in a cluster are, 0 being independent and 1 being identical. The ICC is 0.031, which is nearly the same as the ICC for systolic blood pressure (0.030). This further supports the theory that obserations within a cluster have some correlation.

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [25]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})


In [11]:
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI", groups="group", 
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC is now 0.030. The variables we added to the model did not help to explain the correlation between two values in the same cluster. 

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [13]:
female = da.query('RIAGENDRx == "Female"')
male = da.query('RIAGENDRx == "Male"')

In [18]:
# Fit a marginal linear model using GEE to handle dependent data
modelf = sm.GEE.from_formula("BPXDI1 ~ BMXBMI + RIDAGEYR",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=female)
resultf = modelf.fit()

modelm = sm.GEE.from_formula("BPXDI1 ~ BMXBMI + RIDAGEYR",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=male)
resultm = modelm.fit()

# Compare Results
x = pd.DataFrame({"Female_params": resultf.params, "Female_SE": resultf.bse,
                  "Male_params": resultm.params, "Male_SE": resultm.bse})
x = x[["Female_params", "Female_SE", "Male_params", "Male_SE"]]
print(x)

           Female_params  Female_SE  Male_params   Male_SE
Intercept      65.750510   1.347957    66.894894  1.750734
BMXBMI          0.132351   0.030256     0.265171  0.054721
RIDAGEYR       -0.019222   0.019262    -0.066136  0.016754


__Q3a.__ What do you learn by comparing these two fitted models?

On average across all clusters, women have a lower intercept of diastolic blood pressure than men. BMI and age appear to have stronger impacts on diastolic pressure within males rather than females. 

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [26]:
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})


In [35]:
# Random Intercept by Group
model = sm.MixedLM.from_formula(
    formula = "BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + DMDEDUC2x", 
    groups="group", 
    re_formula = "1", ## intercept
    data=da.dropna())

result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                MixedLM    Dependent Variable:    BPXDI1     
No. Observations:     5100       Method:                REML       
No. Groups:           30         Scale:                 154.2352   
Min. group size:      105        Likelihood:            -20111.9561
Max. group size:      226        Converged:             Yes        
Mean group size:      170.0                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                65.646    1.032 63.631 0.000 63.624 67.668
RIAGENDRx[T.Male]         2.755    0.351  7.855 0.000  2.068  3.443
DMDEDUC2x[T.HS]          -1.093    0.521 -2.098 0.036 -2.115 -0.072
DMDEDUC2x[T.SomeCollege] -0.428    0.484 -0.883 0.377 -1.377  0.521
DMDEDUC2x[T.lt9]         -0.955    0.641 -1.488 0.137 -2.212  0.303
DMDEDUC2x[T.x9_11]       -0.219    0.630 -0.347 0.729 -1.454  1.016
RIDAGEYR                 -0.039    0.010 -3.871 0.000 -0.059 -0.019
BMXBMI                    0.186    0.026  7.283 0.000  0.136  0.236
group Var                 4.170    0.108                           
===================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The group variance is estimated to be 4.170, meaning the random effects between any two groups would differ on average by around `np.sqrt(2*4.170)` = 2.88. This means the clusters appear to be sizably different to each other.


__Q4b:__ Include a random intercept for age, and describe your findings.

In [65]:
# Center the age variable around mean
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

# Random Intercept by Group & Age (??)
model = sm.MixedLM.from_formula(
    formula = "BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + DMDEDUC2x", 
    groups="group", 
    re_formula = "1", ## how do I add age without also varying by slope?
    data=da.dropna())

result = model.fit()
result.summary() 

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                MixedLM    Dependent Variable:    BPXDI1     
No. Observations:     5100       Method:                REML       
No. Groups:           30         Scale:                 154.2358   
Min. group size:      105        Likelihood:            -20112.8450
Max. group size:      226        Converged:             Yes        
Mean group size:      170.0                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                63.719    0.925 68.917 0.000 61.907 65.531
RIAGENDRx[T.Male]         2.754    0.351  7.850 0.000  2.066  3.441
DMDEDUC2x[T.HS]          -1.096    0.521 -2.103 0.035 -2.118 -0.075
DMDEDUC2x[T.SomeCollege] -0.428    0.484 -0.883 0.377 -1.377  0.522
DMDEDUC2x[T.lt9]         -0.976    0.642 -1.521 0.128 -2.234  0.282
DMDEDUC2x[T.x9_11]       -0.224    0.630 -0.355 0.723 -1.459  1.012
age_cen                  -0.037    0.010 -3.632 0.000 -0.057 -0.017
BMXBMI                    0.186    0.026  7.275 0.000  0.136  0.236
group Var                 4.491    0.114                           
===================================================================

"""

In [66]:
result.random_effects

{1191: group   -0.133417
 dtype: float64, 1192: group   -2.547417
 dtype: float64, 1201: group   -0.241045
 dtype: float64, 1202: group    2.069808
 dtype: float64, 1211: group   -1.715873
 dtype: float64, 1212: group    1.162925
 dtype: float64, 1221: group    2.899944
 dtype: float64, 1222: group    0.857928
 dtype: float64, 1231: group    1.395054
 dtype: float64, 1232: group    1.427905
 dtype: float64, 1241: group   -6.25337
 dtype: float64, 1242: group    2.399963
 dtype: float64, 1251: group   -0.71956
 dtype: float64, 1252: group    1.504288
 dtype: float64, 1261: group    0.348723
 dtype: float64, 1262: group   -1.16488
 dtype: float64, 1271: group   -0.530355
 dtype: float64, 1272: group   -1.719174
 dtype: float64, 1281: group   -0.710547
 dtype: float64, 1282: group    1.364895
 dtype: float64, 1291: group    0.087334
 dtype: float64, 1292: group    0.167946
 dtype: float64, 1301: group   -1.807216
 dtype: float64, 1302: group   -1.666625
 dtype: float64, 1311: group   -2.1